# 遗传算法解决旅行商问题。（TSP problem)
the problem is from this site [TSP problem](http://blog.csdn.net/larry233/article/details/50847387)


1. 定义基因序列 gene
2. 定义判定函数 $F = distance $
3. 定义遗传数量（存活数量判定函数）$ P = \frac{F_i}{\sum_{j=1}^NF_j} $
4. 定义交叉
5. 定义变异
6. 获得下一代

got it~

In [15]:
import pandas as pd
import numpy as np
import random
import math
import time

In [16]:
# set the cities number 1 2 3 4, and get the transfer matrix
T = np.array([[0,2,6,5],[2,0,4,4],[6,4,0,2],[5,4,2,0]])
N = 10**3
q_mutation = 0.1
q_wantsex = 1

In [17]:
def F(T,gene):
    # the judgement Function = \sum{distance}
    distance = []
    for i in range(len(gene) - 1):
        distance.append(T[gene[i]][gene[i+1]])
    output = sum(distance) + T[gene[0]][gene[-1]]
    return output

def FP(generation):
    # count the total F and P for each generation
    cur_F = []
    for i in range(len(generation)):
        cur_F.append(F(T,generation[i])) # give each single routes a F to judge it
    cur_F_ar = np.array(cur_F)
    sigma_F = sum(cur_F_ar)
    P = []
    for i in range(len(cur_F)):
        P.append(np.array(cur_F_ar[i])/sigma_F)
    P_ar = np.array(P)
    # set the max P to 1
    P_ar_fil = P_ar*(1/max(P_ar))
    return generation, cur_F_ar, P_ar_fil

In [18]:
def firstgeneration(N,T):
    # set the initial gene series.
    # numbers of routes = N
    gene_lens = len(T[1]) # number of cities
    cur_generation = []
    cur_F = []
    for i in range(N):
        gene_single = list(range(gene_lens))
        random.shuffle(gene_single)
        cur_generation.append(gene_single)
        cur_F.append(F(T,gene_single)) # give each single routes a F to judge it

    cur_F_ar = np.array(cur_F)
    sigma_F = sum(cur_F_ar)
    P = []
    for i in range(N):
        P.append(np.array(cur_F_ar[i])/sigma_F)
    P_ar = np.array(P)
    # set the max P to 1
    P_ar_fil = P_ar*(1/max(P_ar))
    return cur_generation, cur_F_ar, P_ar_fil

In [19]:
def nextgeneration(cur_generation,T,cur_P):
    # the good one lives and they will have the chance to have sex
    # suppose every year they want to have sex.
    N = len(cur_generation)
    new_generation = []
    new_F = []
    new_N = 0
    for i in range(N):
        u = random.random() # sample from 0~1
        if u <= cur_P[i]:
            
            new_generation.append(cur_generation[i])
            new_N += 1
            new_F.append(F(T,cur_generation[i]))

        else:
            continue

    new_F_ar = np.array(new_F)
    sigma_F = sum(new_F_ar)
    P = []
    for i in range(new_N):
        P.append(np.array(new_F_ar[i])/sigma_F)
    P_ar = np.array(P)
    # set the max P to 1
    P_ar_fil = P_ar*(1/max(P_ar))

    return new_generation, new_F_ar, P_ar_fil

In [20]:
def mutation(generation,q):
    # some of the genes switch 2 of its gene
    max_figure = len(generation[0])
    cards = list(range(max_figure))
    for i in range(len(generation)):
        u = random.random()
        if u <= q:
            random.shuffle(cards)
            t1,t2 = cards[0:2]
            generation[i][t1],generation[i][t2] = generation[i][t2],generation[i][t1]
        else:
            continue
    return generation

In [21]:
def cross(generation,q):
    wantsex = []
    for i in range(len(generation)):
        u = random.random()
        if u <= q:
            wantsex.append(generation[i])
    random.shuffle(wantsex)
    for i in range(math.floor(len(wantsex)/2)):
        loc = random.randint(0,len(wantsex[i])-1)
        
        s1_loc = wantsex[2*i].index(wantsex[2*i + 1][loc]) 
        s2_loc = wantsex[2*i + 1].index(wantsex[2*i][loc])
        # 把原来的这两个选定的数交换
        wantsex[2*i][loc],wantsex[2*i+1][loc] = wantsex[2*i+1][loc],wantsex[2*i][loc] 
        # 接下来对调他们本来位置上的这两个值。
        wantsex[2*i][s1_loc],wantsex[2*i+1][s2_loc] = wantsex[2*i+1][s2_loc],wantsex[2*i][s1_loc]
        
    new_generation = generation + wantsex
    return new_generation

In [22]:
start_time = time.time()

yrs = 10
cur_generation,cur_F,cur_P = firstgeneration(N,T)

for i in range(yrs):
    new_generation,new_F,new_P = nextgeneration(cur_generation,T,cur_P)
    new_generation = cross(new_generation,q_wantsex)
    new_generation = mutation(new_generation,q_mutation)
    cur_generation,cur_F,cur_P = FP(new_generation)
    print(len(cur_generation),len(cur_P))

FF = cur_F.tolist()
location = FF.index(min(FF))
best_route = cur_generation[location]
print(best_route,min(FF))

print(time.time() - start_time)

1628 1628
2582 2582
4122 4122
6554 6554
10570 10570
17304 17304
28094 28094
45644 45644
73924 73924
120920 120920
[0, 3, 2, 1] 13
2.810149908065796
